In [1]:
import os
os.chdir('C:/Users/Bibek/OneDrive/JupyterFiles/OptimizationLP/Data')

In [2]:
import pandas as pd
from pulp import *

### Read the given nutrition dataset into a Pandas DataFrame object
Note we are reading only the first 64 rows with `nrows=64` argument because we just want to read all the nutrients informtion and not the maximum/minimum bounds in the dataset. We will enter those bounds in the optimization problem separately.

In [3]:
#df = pd.read_excel("dietForChapter.xls",nrows=6) # example for book chapter

### Show first 5 rows of the dataset

In [27]:
data = {'Foods' : ['Carrots', 'Potatoes', 'Tomatoes', 'Meat', 'Rice', 'Peas'],
      'PricePerUnit' : [0.4, 0.5, 0.75, 1.5, 0.35, 0.4],
      'Calories' : [100, 1000, 200, 200, 1000, 150],
      'Protein' : [8, 8, 16, 40, 12, 24],
      'Vitamin' : [8, 4, 6, 2, 1, 2],
      'Fiber' : [12, 3, 8, 3, 2, 15] }

df = pd.DataFrame(data, columns = ['Foods', 'PricePerUnit','Calories','Protein','Vitamin','Fiber'])

In [28]:
df.head(10)

,Foods,PricePerUnit,Calories,Protein,Vitamin,Fiber
0,Carrots,0.40,100,8,8,12
1,Potatoes,0.50,1000,8,4,3
2,Tomatoes,0.75,200,16,6,8
3,Meat,1.50,200,40,2,3
4,Rice,0.35,1000,12,1,2
5,Peas,0.40,150,24,2,15


### Create the `PuLP` problem variable. Since it is a cost minimization problem, we need to use `LpMinimize`

In [5]:
# Create the 'prob' variable to contain the problem data
prob = LpProblem("Simple_Diet_Problem",LpMinimize)

### Create a list of food items from the dataset

In [6]:
# Creates a list of the Ingredients
food_items = list(df['Foods'])

In [7]:
print("So, the food items to consdier, are\n"+"-"*100)
for f in food_items:
    print(f,end=', ')

So, the food items to consdier, are
----------------------------------------------------------------------------------------------------
Carrots, Potatoes, Tomatoes, Meat, Rice, Peas, 

In [8]:
#food_items[1]

### Create a dictinary of costs for all food items

In [9]:
costs = dict(zip(food_items,df['PricePerUnit']))

In [10]:
costs

{'Carrots': 0.4,
 'Potatoes': 0.5,
 'Tomatoes': 0.75,
 'Meat': 1.5,
 'Rice': 0.35,
 'Peas': 0.4}

### Create a dictionary of calories for all food items

In [11]:
calories = dict(zip(food_items,df['Calories']))
protein = dict(zip(food_items,df['Protein']))
vitamin = dict(zip(food_items,df['Vitamin']))
fiber = dict(zip(food_items,df['Fiber']))

### Create a dictionary of food items with lower bound

In [12]:
# A dictionary called 'food_vars' is created to contain the referenced Variables
food_vars = LpVariable.dicts("Food",food_items,0,cat='Continuous')

In [13]:
food_vars

{'Carrots': Food_Carrots,
 'Potatoes': Food_Potatoes,
 'Tomatoes': Food_Tomatoes,
 'Meat': Food_Meat,
 'Rice': Food_Rice,
 'Peas': Food_Peas}

### Adding the objective function to the problem

In [14]:
# The objective function is added to 'prob' first
prob += lpSum([costs[i]*food_vars[i] for i in food_items]), "Total Cost of lunch pack"

### Adding the constraints to the problem


In [15]:
# Calory
prob += lpSum([calories[f] * food_vars[f] for f in food_items]) >= 800.0, "CaloriesMinimum"
prob += lpSum([calories[f] * food_vars[f] for f in food_items]) <= 1400.0, "CalorieslMaximum"

# Protein
prob += lpSum([protein[f] * food_vars[f] for f in food_items]) >= 50.0, "ProteinMinimum"
prob += lpSum([protein[f] * food_vars[f] for f in food_items]) <= 90.0, "ProteinMaximum"

# Vitamin
prob += lpSum([vitamin[f] * food_vars[f] for f in food_items]) >= 10.0, "VitaminMinimum"
prob += lpSum([vitamin[f] * food_vars[f] for f in food_items]) <= 15.0, "VitaminMaximum"

# Fiber
prob += lpSum([fiber[f] * food_vars[f] for f in food_items]) >= 15.0, "FiberMinimum"
prob += lpSum([fiber[f] * food_vars[f] for f in food_items]) <= 25.0, "FiberMaximum"


### Writing problem data to a `.lp` file

In [16]:
# The problem data is written to an .lp file
prob.writeLP("SimpleDietProblem.lp")

[Food_Carrots, Food_Meat, Food_Peas, Food_Potatoes, Food_Rice, Food_Tomatoes]

### Run the solver

In [17]:
# The problem is solved using PuLP's choice of Solver
prob.solve()

1

### Print the problem solution status `'optimal'`, `'infeasible'`, `'unbounded'` etc...

In [18]:
# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

Status: Optimal


### Scan through the problem variables and print out only if the variable quanity is positive i.e. it is included in the optimal solution

In [19]:
print("Therefore, the optimal (least cost) lunch pack should contain\n"+"-"*110)
for v in prob.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)

Therefore, the optimal (least cost) lunch pack should contain
--------------------------------------------------------------------------------------------------------------
Food_Carrots = 0.843907
Food_Meat = 0.266491
Food_Peas = 0.785647
Food_Rice = 1.14446


### Print the optimal diet cost

In [20]:
print("The total cost of this lunch pack is: ${}".format(round(value(prob.objective),2)))

The total cost of this lunch pack is: $1.45


In [33]:
#for v in prob.variables():
    #print(v.name, "=", v.varValue)